# Tomography reconstruction for PUP_FF_SOFC_hires3_S2

## Objective
Sub-micron resolution with high-energy X-ray tomography

## Sample
* SOFC sample (Cathode, Electrolyte, and Anode)
* First complete/valid scan out of the total seven

## Experiment apparatus
* $\omega \in [-180^\text{o}, 180^\text{o}]$ with $\dot{\omega} = 0.1^\text{o}$
* image resolutoin: $0.2 \mu m / \text{pix}$

In [5]:
import numpy as np
from IPython.display import IFrame

# SVD-based noise reduction

Singular value decompositionv (SVD) is a powerful tool that is commonly used for:
* noise reduction for realtime image stream
* lossy images compression 
* feature detection. 

In this section, the application of SVD based image enhancement for __tomography reconsutrction__ is investigated.

In [4]:
def svd_enhance(img, eigen_cut=20):
    U, S, V = np.linalg.svd(img, full_matrices=True)
    eigen_cut = min(eigen_cut, U.shape[1], V.shape[0]) 
    return np.dot(U[:,:eigen_cut]*S[:eigen_cut], V[:eigen_cut,:])

> Simple example demonstrating eigen feature (eigenimg)

![eigDemo](imgs/eignimg_2628.gif "eig_400")

## SVD enhanced projections with various n_eig

Left= original image , middle= eigen space, right= reconstructed from reduced eigen space (SVD method)

| `n_eig = 400`   | `n_eig = 300`    |
| -----------   | -------------  |
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | 

| `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------:|
|![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

## SVD enhanced projections with various n_eig

Left= original image , middle= eigen space, right= reconstructed from reduced eigen space (SVD method)

| `n_eig = 40`  | `n_eig = 20` |
| :-----:|:-----------: |
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | 

| `n_eig = 10`    | `n_eig = 5`  |
| :-------------:| :-----:|
|![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |

## SVD enhanced projections with various n_eig

* The first __20__ eigen-vectors are sufficient in capturing the main features in each image.
* $F(I_{i,j}) = I_{i,j}^2$ is use to further suppress the noisy background. 

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | ![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | ![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |


In [ ]:
s